In [1]:
from selenium import webdriver
import time
import pandas as pd 

In [2]:
# Esta função cria um dataframe com as informações dentro da página de busca do datajus

# datajus/diarios/

# title = titulo do artigo
# publisher = publicante do artigo
# body = conteúdo da página encontrada com a keyword referenciada
# link = link do artigo 

def page_dataframe():
    df = pd.DataFrame()

    titleL = []
    publisherL = []
    bodyL = []
    linkL = []
    
    for i in range(1,10):
        title = driver.find_element_by_xpath("//*[@id='app-root']/div/div/div[1]/div[2]/div[2]/div[{}]/div/h2".format(i))
        publisher = driver.find_element_by_xpath("//*[@id='app-root']/div/div/div[1]/div[2]/div[2]/div[{}]/div/div[1]".format(i))
        body = driver.find_element_by_xpath("//*[@id='app-root']/div/div/div[1]/div[2]/div[2]/div[{}]/div/div[2]".format(i))
        link = driver.find_element_by_xpath("//*[@id='app-root']/div/div/div[1]/div[2]/div[2]/div[{}]/div/h2/a".format(i))
        link_text = link.get_attribute("href")
        
        titleL.append(title.text)
        publisherL.append(publisher.text)
        bodyL.append(body.text)
        linkL.append(link_text)
    
    df["title"] = titleL
    df["url"] = linkL
    df["publisher"] = publisherL
    df["body"] = bodyL
    
    return df, linkL

In [3]:
options = webdriver.ChromeOptions()
options.add_argument("--start-maximized")

path = "chromedriver.exe"

driver = webdriver.Chrome(executable_path=path, options=options)
time.sleep(3)

In [4]:
# esta celula ainda está sujeita a alteração, já que a KEYWORD pode ser alterado

# dentro de KEYWORD, se coloca a palavra a ser pesquisada
# para usar mais de um termo em KEYWORD, usar mesmas regras do google:
# "covid +suspensão" (encontra artigos que contenham o texto "covid" e também o texto "suspensão")

# filt é a ordem em que os resultados serão mostrados (não é tão relevante já que pegaremos info em massa)
url = 'https://www.jusbrasil.com.br/diarios/'

keyword = "covid+fechamento"
filt = "data&l=1dia"


In [5]:
# loop para entrar em cada uma das páginas do datajus

# Este loop ainda não está completo, por isso deixamos o range baixo (apenas para retirar o dataframe da primeira pagina)

for i in range(1,2):
    page = i
    
    search = "busca?q="+keyword+"&p="+str(page)+"o="+filt
    driver.get(url+search)
    
    time.sleep(2)
    
    df, linkL = page_dataframe()
    
    
    #loop dentro de cada um dos links que estão dentro da página, para entrar no artigo e extrair os dados
    count=0
    for link in linkL:
        time.sleep(2)
        driver.get(link)
        body = driver.find_element_by_xpath("//*[@id='app-root']/div/div/div[1]/div/div[2]/div/div[1]")
        
        ser = pd.Series()
        ser["body"] = body.text
        
        df.loc[count]["body"] = ser["body"]
        count=count+1

In [6]:
df.to_csv(r'vraul.csv')